# Troop Booth Signups Analysis

This notebook analyzes the troop booth signup data from the file **data/2025_booth_signups_trimmed.csv**. The analysis includes:

- **Data Loading & Preparation:** Reading the CSV file and converting time columns to datetime objects.
- **Aggregation & Descriptive Statistics:** Grouping by troop to count the number of signup events, computing summary statistics, and determining the interquartile range (IQR).
- **Outlier Detection:** Defining outliers as troops with signup counts below Q1 - 1.5×IQR or above Q3 + 1.5×IQR.
- **Visualization:** Creating a horizontal bar chart of the signup counts for all troops with horizontal reference lines for the thresholds and highlighting outlier troops in a distinct color.
- **Report Summary:** A textual summary of the analysis and its significance.

Each row in the data represents one booth signup event. The final chart and report help identify which troops are operating outside of the norm.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set up plotting style and figure size defaults
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Load the Data

The data file is located at **data/2025_booth_signups_trimmed.csv**. It contains a header with the following columns:

- **Troop**
- **Troop Email**
- **Slot Start Time**
- **Slot End Time**
- **When Selected Date**
- **When Selected Time**
- **User Selecting**

Let's load the file into a DataFrame.

In [ ]:
# Define the data file path
data_file = 'data/2025_booth_signups_trimmed.csv'

# Load the CSV file (assuming it has headers)
df = pd.read_csv(data_file)

# Preview the data
print("Data preview:")
df.head()

## 2. Data Preparation

We convert the time columns to datetime objects. (The times in the file are formatted as `YYYY/MM/DD HH:MM:SS`.)

For this analysis we focus on counting the number of signup events per troop.

In [ ]:
# Convert time columns to datetime objects
df['Slot Start Time'] = pd.to_datetime(df['Slot Start Time'], format='%Y/%m/%d %H:%M:%S', errors='coerce')
df['Slot End Time']   = pd.to_datetime(df['Slot End Time'], format='%Y/%m/%d %H:%M:%S', errors='coerce')

# Convert the When Selected columns as well
df['When Selected Date'] = pd.to_datetime(df['When Selected Date'], format='%Y/%m/%d %H:%M:%S', errors='coerce')
df['When Selected Time'] = pd.to_datetime(df['When Selected Time'], format='%Y/%m/%d %H:%M:%S', errors='coerce')

# Display DataFrame info to verify conversions
df.info()

## 3. Aggregation & Descriptive Statistics

We count the number of booth signup events for each troop. Since each row represents one event, we group by the **Troop** column and count the rows. We then compute descriptive statistics and calculate the interquartile range (IQR) to determine the lower and upper thresholds for outlier detection.

In [ ]:
# Group by Troop and count the number of signup events
troop_counts = df.groupby('Troop').size().reset_index(name='Num_Booths')

# Sort the aggregated data by signup count
troop_counts.sort_values('Num_Booths', ascending=False, inplace=True)
print("Booth signup counts per troop:")
print(troop_counts)

# Compute descriptive statistics
desc_stats = troop_counts['Num_Booths'].describe()
print("\nDescriptive statistics:")
print(desc_stats)

# Calculate quartiles and IQR
Q1 = troop_counts['Num_Booths'].quantile(0.25)
Q3 = troop_counts['Num_Booths'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"\nQ1: {Q1}, Q3: {Q3}, IQR: {IQR}")
print(f"Lower bound: {lower_bound}, Upper bound: {upper_bound}")

## 4. Outlier Detection

Using the IQR method, we define outliers as troops with signup counts below **lower_bound** or above **upper_bound**. We display these outliers.

In [ ]:
# Identify outlier troops
outliers = troop_counts[(troop_counts['Num_Booths'] < lower_bound) | (troop_counts['Num_Booths'] > upper_bound)]
print("\nOutlier troops (by number of booth signups):")
print(outliers)

## 5. Bar Chart: All Troops with Outlier Highlighting

The following bar chart displays the number of booth signup events for each troop. Outlier troops (as determined by the IQR thresholds) are color‑coded in red, while the other troops are shown in blue. Horizontal reference lines for the lower and upper bounds are also drawn.

In [ ]:
# Create a bar chart for all troops, highlighting outliers

# Color code: red for outliers, blue for non-outliers
colors = ['red' if (x < lower_bound or x > upper_bound) else 'blue' for x in troop_counts['Num_Booths']]

plt.figure(figsize=(12,6))
plt.bar(troop_counts['Troop'], troop_counts['Num_Booths'], color=colors)

plt.xlabel('Troop')
plt.ylabel('Number of Booth Signups')
plt.title('Booth Signups per Troop with Outlier Highlighting')

# Draw horizontal lines for the lower and upper bounds
plt.axhline(lower_bound, color='orange', linestyle='--', linewidth=2, label=f'Lower Bound ({lower_bound})')
plt.axhline(upper_bound, color='green', linestyle='--', linewidth=2, label=f'Upper Bound ({upper_bound})')

plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 6. Bar Chart: Outlier Troops Only

The chart below focuses solely on the outlier troops. It is a horizontal bar chart displaying only those troops that fall outside the normal range (i.e. below the lower bound or above the upper bound).

In [ ]:
# Filter the aggregated data to include only outlier troops
outlier_troops = troop_counts[(troop_counts['Num_Booths'] < lower_bound) | (troop_counts['Num_Booths'] > upper_bound)]

# Sort the outlier data
outlier_troops = outlier_troops.sort_values('Num_Booths', ascending=True)

# Create a horizontal bar chart for outlier troops
plt.figure(figsize=(10, 6))
bars = plt.barh(outlier_troops['Troop'], outlier_troops['Num_Booths'], color='salmon')

# Add labels to each bar
for i, (troop, count) in enumerate(zip(outlier_troops['Troop'], outlier_troops['Num_Booths'])):
    plt.text(count + 0.5, i, str(count), va='center', fontsize=10)

# Draw vertical lines indicating the lower and upper thresholds
plt.axvline(lower_bound, color='orange', linestyle='--', linewidth=2, label=f'Lower Bound ({lower_bound})')
plt.axvline(upper_bound, color='green', linestyle='--', linewidth=2, label=f'Upper Bound ({upper_bound})')

plt.xlabel('Number of Booth Signups')
plt.title('Outlier Troops in Booth Signups')
plt.legend()
plt.tight_layout()
plt.show()

## 7. Analysis Report

Below is a summary of the analysis based on the aggregated data:

### Overall Statistics

- **Total Troops Analyzed:** {total_troops}
- **Mean Booth Signups:** {mean_signups:.2f}
- **Median Booth Signups:** {median_signups:.2f}
- **Minimum Booth Signups:** {min_signups}
- **Maximum Booth Signups:** {max_signups}
- **Standard Deviation:** {std_signups:.2f}

### IQR-Based Thresholds

- **Q1:** {Q1}
- **Q3:** {Q3}
- **IQR:** {IQR}
- **Lower Bound:** {lower_bound}
- **Upper Bound:** {upper_bound}

Troops with booth signup counts outside the interval [lower bound, upper bound] are considered outliers.

### Outlier Details

**High Outliers:** Troops with counts greater than {upper_bound}.
**Low Outliers:** Troops with counts less than {lower_bound}.

These outliers might indicate either exceptionally proactive troops (if high) or under‐participating troops (if low). Further review with troop leaders is recommended to determine if these are due to operational differences or data entry errors.

### Significance

Understanding which troops fall outside the typical range can help target additional support, resources, or training. High outliers may be models for best practices, while low outliers may need further engagement or investigation.

In [ ]:
# Generate a textual report using the computed statistics

# Calculate overall values from troop_counts and desc_stats
total_troops = len(troop_counts)
mean_signups = desc_stats['mean']
median_signups = desc_stats['50%']
min_signups = desc_stats['min']
max_signups = desc_stats['max']
std_signups = desc_stats['std']

report = f"""
# Troop Booth Signups Analysis Report

**Data Source:** data/2025_booth_signups_trimmed.csv
**Total Troops Analyzed:** {total_troops}

## Overall Statistics
- Mean Booth Signups: {mean_signups:.2f}
- Median Booth Signups: {median_signups:.2f}
- Minimum Booth Signups: {min_signups}
- Maximum Booth Signups: {max_signups}
- Standard Deviation: {std_signups:.2f}

## IQR-Based Thresholds
- Q1: {Q1}
- Q3: {Q3}
- IQR: {IQR}
- Lower Bound: {lower_bound}
- Upper Bound: {upper_bound}

Troops with booth signup counts outside the range [{lower_bound}, {upper_bound}] are considered outliers.

## Outlier Details

### High Outliers (Counts > {upper_bound})
"""

if not high_outliers.empty:
    for index, row in high_outliers.iterrows():
        deviation = row['Num_Booths'] - upper_bound
        report += f"- Troop {row['Troop']}: {row['Num_Booths']} signups ( +{deviation:.2f} above the upper bound)\n"
else:
    report += "No high outliers detected.\n"

report += "\n### Low Outliers (Counts < {lower_bound})\n"

if not low_outliers.empty:
    for index, row in low_outliers.iterrows():
        deviation = lower_bound - row['Num_Booths']
        report += f"- Troop {row['Troop']}: {row['Num_Booths']} signups ( -{deviation:.2f} below the lower bound)\n"
else:
    report += "No low outliers detected.\n"

report += "\n## Summary and Recommendations\n"
report += "Review the above outlier troops to determine if they represent exceptional performance or require additional support. High outliers might indicate proactive management or possible data anomalies, while low outliers could signal under-participation or scheduling challenges. Further investigation and follow-up with the respective troop leaders is recommended.\n"

print(report)